<a href="https://colab.research.google.com/github/yerinsally/4J/blob/main/12_KLUE_NLI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install -qq transformers==4.16.0 datasets sentencepiece
# ! pip install -qq Korpora

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 50.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.5 MB/s eta 0:0

In [3]:
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1KG_cdxFYy-w1UTA9hOsASk87_B_aBXUR" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1KG_cdxFYy-w1UTA9hOsASk87_B_aBXUR" -o open.zip
!unzip open.zip
!rm -rf open.zip cookie
!nvidia-smi

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1773k  100 1773k    0     0  1613k      0  0:00:01  0:00:01 --:--:-- 1613k
Archive:  open.zip
  inflating: open/sample_submission.csv  
  inflating: open/test_data.csv      
  inflating: open/train_data.csv     
Mon May  8 05:32:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+==

In [4]:
import os
import pandas as pd
import numpy as np

import random
from tqdm import tqdm
import gc

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AutoModel, AutoConfig, AutoTokenizer, AdamW
from transformers.optimization import get_linear_schedule_with_warmup
from datasets import load_dataset

# from Korpora import KorNLIKorpus

RANDOM_SEED = 42

In [5]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  
    torch.backends.cudnn.deterministic = True  
    torch.backends.cudnn.benchmark = True  

seed_everything(RANDOM_SEED)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [6]:
# MODEL_NAME = "klue/roberta-large"
# MODEL_NAME = "monologg/koelectra-base-v3-discriminator"
MODEL_NAME = "kykim/electra-kor-base"
# MODEL_NAME = "Jinhwan/krelectra-base-mecab"
# MODEL_NAME = "skt/kobert-base-v1"
# NUM_SAMPLES = 10000
LEARNING_RATE = 5e-5
BATCH_SIZE = 32
MAX_LENGTH = 216
NUM_FOLDS = 5
EPOCHS = 1

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
train_df = pd.read_csv("/content/open/train_data.csv")
test_df = pd.read_csv("/content/open/test_data.csv")

In [ ]:
# corpus = KorNLIKorpus()
# corpus.xnli_dev[0]

In [ ]:
# label_to_num = {"entailment": 0, "neutral": 1, "contradiction": 2}
# p_kornli, h_kornli = [], []
# labels = []
# for idx in tqdm(range(len(corpus.xnli_dev))):
#     p_kornli.append(corpus.xnli_dev[idx].text)
#     h_kornli.append(corpus.xnli_dev[idx].pair)
#     labels.append(label_to_num[corpus.xnli_dev[idx].label])

# for index in tqdm(range(len(corpus.xnli_test))):
#     p_kornli.append(corpus.xnli_test[index].text)
#     h_kornli.append(corpus.xnli_test[index].pair)
#     labels.append(label_to_num[corpus.xnli_test[index].label])

# p_kornli[0], h_kornli[0], labels[0], len(labels)

In [9]:
nli_train = load_dataset("klue", "nli", split="train")
nli_validation = load_dataset("klue", "nli", split="validation")
nli_train = pd.DataFrame(nli_train)
nli_validation = pd.DataFrame(nli_validation)
print(len(nli_train)+len(nli_validation))
nli_data = pd.concat([nli_train, nli_validation], axis=0)
# nli_data

Generating train split:   0%|          | 0/24998 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/nli/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e. Subsequent calls will reuse this data.


27998


In [10]:
class BertLSTM(nn.Module):
    def __init__(self, model_name):
        super(BertLSTM, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.hidden_dim = self.bert.config.hidden_size
        self.lstm = nn.LSTM(input_size=self.hidden_dim,
                            hidden_size=self.hidden_dim,
                            num_layers=1,
                            bidirectional=True,
                            batch_first=True)
        self.linear = nn.Linear(in_features=2*self.hidden_dim, out_features=3)

    def forward(self, input_ids, attention_mask):
        out = self.bert(input_ids=input_ids, attention_mask=attention_mask)["last_hidden_state"] 
        _, (hidden, _) = self.lstm(out) 
        output = torch.cat([hidden[0], hidden[1]], dim=1)
        output = self.linear(output)
        return output

# self.bert : 입력 문장을 처리하기 위한 BERT 모델 객체 생성
# self.hidden_dim : BERT 모델의 hidden size 저장
# self.lstm : 레이어 정의, 입력차원 = BERT의 hidden size, 출력차원 = hidden size*2
# self.linear : 출력층 정의, 입력차원 = BiLSTM 출력차원*2, 출력차원 = 3(분류할 관계 유형 개수)
# forward 메서드 : 입력 문장의 BERT 임베딩 계산
## input_ids, attention_mask : 입력 문장 토큰화한 결과
## forward : BiLSTM의 forward 방향의 hidden state를 concat하여 최종 출력 벡터 생성
## self.linear : 최종 출력 벡터를 선형 레이어에 입력하여 예측값 출력

In [11]:
def get_tokenized(tokenizer, premise, hypothesis):
    return tokenizer(
        premise,
        hypothesis,
        max_length=216,
        truncation=True,
        padding=True,
        add_special_tokens=True,
        return_tensors="pt"
    )

In [12]:
class NLIDataset(Dataset):
    def __init__(self, pair_dataset, label):
        self.pair_dataset = pair_dataset
        self.label = label

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
        item['label'] = torch.tensor(self.label[idx])
        return item


class TestDataset(Dataset):
    def __init__(self, pair_dataset):
        self.pair_dataset = pair_dataset

    def __len__(self):
        return len(self.pair_dataset["input_ids"])

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
        return item
# NLI 데이터셋을 정의하는 클래스(NLIDataset와 TestDataset) 구현
# pair_dataset : 입력 문장 쌍 포함하는 딕셔너리, input_ids, attention_mask, token_type_ids 키를 가지며 각각은 BERT의 입력으로 사용됨
# label : 입력 문장 쌍의 레이블을 포함하는 리스트 -> TestDataset에는 해당 키 없음
# __len__ : 데이터셋 크기 반환

In [13]:
def get_datloader(tokenizer, train_data, valid_data):
    train_tokenized = get_tokenized(tokenizer, list(train_data["premise"]), list(train_data["hypothesis"]))
    valid_tokenized = get_tokenized(tokenizer, list(valid_data["premise"]), list(valid_data["hypothesis"]))

    train_dataset = NLIDataset(train_tokenized, list(train_data["label"]))
    valid_dataset = NLIDataset(valid_tokenized, list(valid_data["label"]))

    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, pin_memory=True, shuffle=True)
    valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, pin_memory=True, shuffle=False)

    return train_dataloader, valid_dataloader

In [15]:
def fetch_scheduler(model, train_dataloader):
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer,
                                                    max_lr=LEARNING_RATE,
                                                    steps_per_epoch=len(train_dataloader), 
                                                    epochs=EPOCHS,
                                                    pct_start=0.1)
    return optimizer, scheduler
# 학습 시 learning rate를 조절하는 스케줄러(scheduler)를 생성하는 함수인 fetch_scheduler 정의
# optimizer : adam optimizer 사용, 모델의 파라미터와 학습률을 인자로 받아 optimizer 초기화
# scheduler : OneCycleLR 스케쥴러 사용, 한번의 에폭에서 learning rate 조절
## steps_per_epoch : 한번의 에폭에서 데이터셋을 몇번 미니배치로 분할할지 결정
## pct_start : learning rate 최대값을 설정하는데 걸리는 비율

In [ ]:
# Label Smoothing
# criterion = nn.CrossEntropyLoss()
# optimizer = AdamW(
#     [{"params": [v for k, v in model.named_parameters() if k.startswith("bert")]}, 
#      {"params": [v for k, v in model.named_parameters() if not k.startswith("bert")], "lr": LEARNING_RATE*10}],
#     lr=LEARNING_RATE,
#     no_deprecation_warning=True
# )
# scheduler = get_linear_schedule_with_warmup(
#     optimizer, 
#     num_warmup_steps=0, 
#     num_training_steps=EPOCHS*(len(train_dataloader))
# )
# optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# scheduler = torch.optim.lr_scheduler.OneCycleLR(
#     optimizer,
#     max_lr=LEARNING_RATE,
#     steps_per_epoch=len(train_dataloader), 
#     epochs=EPOCHS,
#     pct_start=0.1
# )

In [16]:
def train(model, optimizer, scheduler, criterion, train_dataloader, valid_dataloader):
    for epoch in range(EPOCHS):
        gc.collect()
        model.train()
        epoch_loss = 0
        steps = 0
        pbar = tqdm(train_dataloader)
        for batch in pbar:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            label = batch["label"].to(device)

            output = model(input_ids, attention_mask)
            loss = criterion(output, label)
            epoch_loss += loss.detach().cpu().numpy()
            steps += 1

            optimizer.zero_grad()
            loss.backward()

            optimizer.step()
            scheduler.step()

            pbar.set_postfix({'loss': epoch_loss/steps, 
                            "lr": optimizer.param_groups[0]["lr"], 
                            "cls_lr": optimizer.param_groups[-1]["lr"]})
            # del loss, input_ids, attention_mask, label, output

        pbar.close()

        with torch.no_grad():
            model.eval()
            all_preds, all_labels = [], []
            for valid_batch in tqdm(valid_dataloader):
                input_ids = valid_batch["input_ids"].to(device)
                attention_mask = valid_batch["attention_mask"].to(device)
                label = valid_batch["label"]

                output = model(input_ids, attention_mask)
                preds = output.argmax(-1).detach().cpu().numpy()
                all_preds.extend(preds)
                all_labels.extend(label)
                # del input_ids, attention_mask, label, output
            accuracy = accuracy_score(all_labels, all_preds)
            print(f"Epoch [{epoch+1}/{EPOCHS}] Accuracy : {accuracy}")

    del train_dataloader, valid_dataloader
    gc.collect()

In [17]:
@torch.no_grad()
def inference(model, test_dataloader):
    model.eval()
    all_preds = []
    for batch in tqdm(test_dataloader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        output = model(input_ids, attention_mask)
        preds = output.argmax(-1).detach().cpu().numpy()
        all_preds.extend(preds)

        # del input_ids, attention_mask, output
    # del test_dataloader
    return all_preds

In [18]:
skf = StratifiedKFold(n_splits=NUM_FOLDS, random_state=RANDOM_SEED, shuffle=True)
num_to_label = {0: "entailment", 1: "neutral", 2: "contradiction"}
fold = 0
for fold_num, (train_idx, valid_idx) in enumerate(skf.split(nli_data[["premise", "hypothesis"]], nli_data["label"])):

    train_data = nli_data.iloc[train_idx]
    valid_data = nli_data.iloc[valid_idx]

    model = BertLSTM(MODEL_NAME).to(device)
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    
    train_dataloader, valid_dataloader = get_datloader(tokenizer, train_data, valid_data)

    optimizer, scheduler = fetch_scheduler(model, train_dataloader)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.2)

    print("="*100)
    print(f"Fold-{fold_num} Training")
    print("="*100)
    train(model, 
          optimizer, 
          scheduler, 
          criterion, 
          train_dataloader, 
          valid_dataloader)

    test_df = pd.read_csv("/content/open/test_data.csv")
    test_tokenized = get_tokenized(
                                    tokenizer,
                                    list(test_df["premise"]), 
                                    list(test_df["hypothesis"]))
    test_dataset = TestDataset(test_tokenized)
    test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

    submission = pd.read_csv("/content/open/sample_submission.csv")
    predictions = inference(model, test_dataloader)
    submission["label"] = predictions

    submission["label"] = submission["label"].apply(lambda x: num_to_label[x])
    submission.to_csv(f"./submission-fold{fold_num}.csv", index=False)

    del model, train_dataloader, valid_dataloader, test_dataloader
    _ = gc.collect()
    torch.cuda.empty_cache()

    fold = fold_num

# 40분 걸림

Downloading:   0%|          | 0.00/620 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/451M [00:00<?, ?B/s]

Some weights of the model checkpoint at kykim/electra-kor-base were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/336k [00:00<?, ?B/s]

...

100%|██████████| 53/53 [00:08<00:00,  6.22it/s]
Some weights of the model checkpoint at kykim/electra-kor-base were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Fold-4 Training


100%|██████████| 175/175 [00:34<00:00,  5.12it/s]


Epoch [1/1] Accuracy : 0.8839078406858367


100%|██████████| 53/53 [00:08<00:00,  6.27it/s]


In [19]:
sub_fold = [f"./submission-fold{f}.csv" for f in range(fold)]
final_df = pd.DataFrame()
for idx, sub in enumerate(sub_fold):
    df = pd.read_csv(sub)
    final_df[idx] = df["label"]

final_label = list(final_df.mode(axis=1)[0])
submission = pd.read_csv("./open/sample_submission.csv")
submission["label"] = final_label
submission.to_csv("./submission.csv", index=False)